In [1]:
import polars as pl
import matplotlib.pyplot as plt
import japanize_matplotlib
import yfinance as yf
from math import ceil

In [2]:
stock_ref = pl.read_csv("../resources/all_stock.csv").select("コード", "銘柄名")
trust_ref = pl.read_csv("../resources/all_trust.csv")

In [5]:
# stocks
df = pl.read_csv("../results/stock/optimized_portfolio.csv").filter(pl.col("Weight") > 0)
df = df.join(stock_ref, left_on="Ticker", right_on="コード").sort("Weight", descending=True)

fig, axes = plt.subplots(ceil(len(df) / 5), 5, figsize = (30, 6* ceil(len(df) / 5)))
axes = axes.flatten()

for i, row in enumerate(df.iter_rows(named=True)):

    ticker = row["Ticker"] +".T"
    ax = axes[i]
    stock_data = yf.download(ticker, interval = "5d", period = "max", progress=False, auto_adjust=True)
    color = plt.cm.copper(row["Weight"])
    stock_data["Close"].plot(ax=ax, title=f"{row['銘柄名']} ({row['Ticker']})", legend=False, color=color)
    ax.set_xlabel("")

fig.tight_layout()
fig.savefig("../results/stock/stock_portfolio_summary.png", dpi=300, bbox_inches='tight')
plt.close()

df.write_csv("../results/stock/stock_portfolio_summary.csv")

In [8]:
# trusts
df = pl.read_csv("../results/trust/optimized_portfolio.csv", schema_overrides={"Ticker":str, "Weight": pl.Float64}).filter(pl.col("Weight") > 0.0)
df = df.join(trust_ref, left_on="Ticker", right_on="協会コード").sort("Weight", descending=True)

fig, axes = plt.subplots(ceil(len(df) / 5), 5, figsize = (30, 6* ceil(len(df) / 5)))
axes = axes.flatten()

for i, row in enumerate(df.iter_rows(named=True)):

    ticker = row["Ticker"]
    ax = axes[i]
    stock_data = pl.read_csv(f"../data/trust/{ticker}.csv").select("年月日", "基準価額").with_columns(pl.col("年月日").str.strptime(pl.Date, format=r"%Y-%m-%d")).sort("年月日")
    color = plt.cm.copper(row["Weight"])
    ax.plot(stock_data["年月日"], stock_data["基準価額"], color=color)
    ax.set_title(f"{row['ファンド名']}")
    ax.tick_params(axis='x', rotation=45)

fig.tight_layout()
fig.savefig("../results/trust/trust_portfolio_summary.png", dpi=300, bbox_inches='tight')
plt.close()

df.write_csv("../results/trust/trust_portfolio_summary.csv")

In [9]:
df = pl.read_csv("../results/emaxis/optimized_portfolio.csv").filter(pl.col("Weight") > 0.0)
df = df.join(trust_ref, left_on="Ticker", right_on="協会コード").sort("Weight", descending=True)

fig, axes = plt.subplots(ceil(len(df) / 5), 5, figsize = (30, 6* ceil(len(df) / 5)))
axes = axes.flatten()

for i, row in enumerate(df.iter_rows(named=True)):

    ticker = row["Ticker"]
    ax = axes[i]
    stock_data = pl.read_csv(f"../data/trust/{ticker}.csv").select("年月日", "基準価額").with_columns(pl.col("年月日").str.strptime(pl.Date, format=r"%Y-%m-%d")).sort("年月日")
    color = plt.cm.copper(row["Weight"])
    ax.plot(stock_data["年月日"], stock_data["基準価額"], color=color)
    ax.set_title(f"{row['ファンド名']}")
    ax.tick_params(axis='x', rotation=45)

fig.tight_layout()
fig.savefig("../results/emaxis/trust_portfolio_summary.png", dpi=300, bbox_inches='tight')
plt.close()

df.write_csv("../results/emaxis/trust_portfolio_summary.csv")